In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

reader = SimpleDirectoryReader(input_dir="/home/dai/35/rag/data/clean_html",
                                  recursive=True)

documents = reader.load_data()
print(len(documents))

node_parser = HTMLNodeParser()
nodes = node_parser.get_nodes_from_documents(documents)

594


In [5]:
node_list = list(nodes)
node_list[2].get_content()

'AayurVidnyaana Kosha is encyclopedia of Ayurveda.\nThe AayurVidnyaana\nKosha window is divided into two frames. The left frame has three tabs,\nContents, Search and New Window. Right frame is for viewing the document.\nAbout AayurVidnyaana Kosha\nAyuSoft encyclopedia elaborates scientific basis,\ntheory and strength of Ayurveda in a dynamic form.\nKey topic of different subjects highlighted with\ntheir clinical applications.\nThis includes information regarding Ayurvedic\nconcepts, current status of Ayurveda and related research articles.\nThe Encyclopedia of AyuSoft includes articles from\neminent people related with the field of Ayurveda.\nThe Articles are focused and classified according\nto the respective subjects to cover all the branches of Ayurveda.\nAyuSoft Encyclopedia provides elaboration of\nvarious topics required for decision support.\nThese are made available in form of guidelines for\npatient assessment, Panchakarma And allied Karma, pathogenesis\nand theory of disease.

In [8]:
from llama_index.embeddings.gemini import GeminiEmbedding
embedding = GeminiEmbedding()
for node in node_list:
    try:
        embedding.get_text_embedding(node.get_content())
    except Exception as e:
        print(len(node.get_content()))
        print(node.metadata)

10468
{'tag': 'p', 'file_path': '/home/dai/35/rag/data/clean_html/Aayurveda_Samhitaa/01_Charaka_Samhitaa/DF_Introduction_Charaka_Samhita.html', 'file_name': 'DF_Introduction_Charaka_Samhita.html', 'file_type': 'text/html', 'file_size': 13579, 'creation_date': '2024-02-13', 'last_modified_date': '2024-02-13', 'last_accessed_date': '2024-02-13'}
0
{'tag': 'u', 'file_path': '/home/dai/35/rag/data/clean_html/Aayurveda_Samhitaa/02_Sus`hruta_Samhitaa/DF_Introduction_Sushruta_Samhitaa.html', 'file_name': 'DF_Introduction_Sushruta_Samhitaa.html', 'file_type': 'text/html', 'file_size': 6517, 'creation_date': '2024-02-13', 'last_modified_date': '2024-02-13', 'last_accessed_date': '2024-02-13'}
19977
{'tag': 'p', 'file_path': '/home/dai/35/rag/data/clean_html/Articles/Chikitsaa/Basics_of_Chikitsaa.htm', 'file_name': 'Basics_of_Chikitsaa.htm', 'file_type': 'text/html', 'file_size': 24391, 'creation_date': '2024-02-13', 'last_modified_date': '2024-02-13', 'last_accessed_date': '2024-02-13'}
0
{'tag

KeyboardInterrupt: 